Dou et al. (2020) model

Agora em Julia versão 1.5.3

PROTOTYPE

This script simulates the model, then estimates the parameters using simulated annealing

In [1]:
using LinearAlgebra, Statistics

# using Distributions, Expectations, NLsolve, Roots, Random, Plots, Parameters


using DataFrames
using BenchmarkTools



In [2]:
using NBInclude
@nbinclude("solve_model.ipynb")

#importou a função solve_tree

solve_tree (generic function with 5 methods)

Functions to simulate the game

In [3]:
function error_vector(data_moments, simulated_moments)
    #two arrays with the data moments and the simulated moments
    
    return (simulated_moments .- data_moments) ./ data_moments
    
end



    

error_vector (generic function with 1 method)

In [4]:
function criterion(S, N, data, W_hat, game_parameters, simulation_parameters, s_W, j_W, Pst_array, Pjt_array)
    
    θs0, θj0, μ = simulation_parameters
    θs0 = Int64(θs0)
    θj0 = Int64(θj0)
      
    ρ = game_parameters[1]
    c0 = game_parameters[3]
    λj = game_parameters[4]
    c1 = Float64(0.015) #calibrated


    Vmax = data[1]
    L = data[2]
    Dj = data[3]
    Ds = (1.0 - Dj)
    
    D = Dj + Ds
    
    #return error message if the data is not escaled
    #retorna mensagem de erro se os dados não estiverem em escala
    @assert(Ds > 0 && D == 1.0)
    
    
    function simulate_moments(S, N, θs0, θj0, μ, λj, s_W=s_W, j_W=j_W, Pst_array=Pst_array, Pjt_array=Pjt_array)
    
    
        Results = loop_simulations(S, N, θs0, θj0, μ, λj)

        #S simulations, 8 moments
        Moments = zeros(S, 8)


        for s in 1:S

            DF = DataFrame()
            DF.payoff_s = Results[s,:,1];
            DF.payoff_j = Results[s,:,2];
            DF.out = Results[s,:,3];
            DF.t = Results[s,:,4];
            DF.observed_proposals = Results[s,:,5];
            DF.Dj = Results[s,:,6];
            DF.Ds = Results[s,:,7];


            DF.outcome = ifelse.(DF.out .==3.0, "R", "L");


            #desfazendo o deslocamento no índice de t
            DF.t = DF.t .- 1.0;

            # DF.incourt = ifelse.(DF.t .> 0.0, "incourt", "precourt");

            INCOURT = filter(DF -> DF.t .> 0.0, DF);
            PRECOURT = filter(DF -> DF.t .<= 0.0, DF);

            #cálculo dos momentos
            #1. avg log number of months between observed proposals incourt

            mm1 = INCOURT
            mm1 = log.(mm1.t .* μ ./ mm1.observed_proposals)
            mm1 = mean(mm1)

            #2. fraction reorganized given that the case went into court

            mm2 = INCOURT
            mm2 = size(filter(mm2 -> mm2.outcome .== "R", mm2),1)/ size(mm2,1)

            #3. ln duration of court cases in months

            mm3 = INCOURT
            mm3 = filter(mm3 -> mm3.t .> 0.0, mm3) #removendo os casos 0 para não poluir a média
            mm3.t = mm3.t .* μ
            mm3 = mean(log.(mm3.t)) #log here uses exp as base, so it's the same as ln

            #4. fraction of cases incourt
            mm4 = size(INCOURT,1) / size(DF, 1)

            #5. avg recovery rate for senior given precourt

            #aqui nós temos Ds, então é mais fácil. Preciso automatizar isso no código quando for tudo escalado

            mm5 = PRECOURT
            mm5.payoff_s = mm5.payoff_s ./ PRECOURT.Ds
            mm5 = mean(mm5.payoff_s)


            #6. avg recovery rate for junior given precourt

            #aqui nós temos Ds, então é mais fácil. Preciso automatizar isso no código quando for tudo escalado

            mm6 = PRECOURT
            mm6.payoff_j = mm6.payoff_j ./ PRECOURT.Dj
            mm6 = mean(mm6.payoff_j)


            #7. junior avg fraction gain given incourt

            mm7 = INCOURT
            mm7 = mean(mm7.payoff_j ./ (mm7.payoff_j .+ mm7.payoff_s )) 

            #8. total recovery rate given incourt

            mm8 = INCOURT
            mm8 = mean( mm8.payoff_s .+ mm8.payoff_j)



            Moments[s,:] .= [mm1, mm2, mm3, mm4, mm5, mm6, mm7, mm8]

        end

        return vec(mean(Moments, dims=1))
    end

    
    
    function loop_simulations(S, N, θs0, θj0, μ, λj, s_W=s_W, j_W=j_W, Pst_array=Pst_array, Pjt_array=Pjt_array)
    
        #S is the number of simulations
        #N is the number of observations

        #no futuro os argumentos podem ser arrays de arrays, assim ele faz o loop para cada cluster

        Results = zeros(S, N, 7)

        #últimas entradas de Results são os valores das dívidas
        Results[:,:, end-1] .= Dj
        Results[:,:, end] .= Ds


        for s in 1:S
            for n in 1:N
                Results[s, n, 1:5] .= simulate_game(θs0, θj0, μ, λj)
            end
        end

        return Results

    end
    
    
    
    function simulate_game(Hs0, Hj0, μ, λj, s_W=s_W, j_W=j_W, Pst_array=Pst_array, Pjt_array=Pjt_array, t=1, grid=100)
    
        hst = Hs0
        hjt = Hj0

        #assumindo que lower bounds nos períodos iniciais são as próprias habilidades iniciais
        lst = hst
        ljt = hjt

        result = zeros(5);

        #number of observed proposals
        observed_proposals = 0.0


        #recovering "T+1" from s_W 
        T = size(s_W,1)


        while(result[1]==0.0 && t < T)


            u = rand()

            if(u < λj)
                propositor = "j"

            else
                propositor = "s"

            end


            #setting the default variables according to the propositor
            Pkt_array, Cont_val, prop_index, respondent_index, m_L, hkt, lkt, hmt, lmt, lk_next, hk_next, hm_next = choose_parameters(propositor, hst, lst, hjt, ljt)


            #proposal ####
            policy, payoff_prop, payment, lm_next = proposal(Pkt_array, t, hkt, lmt)

            #lm_next is the update of the adversary's lower bound
            if(lm_next==grid+2 || policy!= 3.0) #para evitar update de lowerbound quando proponente não propõe reorg
                lm_next = lmt
            else
                lm_next = Int64(lm_next)
            end



            if(policy==3.0)

                observed_proposals += 1.0

                payoff_respondent, answer = answer_reorg(payment, Cont_val, t, hm_next, lm_next, lk_next)


                if(answer==1.0)

                    result[prop_index] = payoff_prop
                    result[respondent_index] = payoff_respondent
                    result[3] = 3.0
                    result[4] = t
                    result[5] = observed_proposals

                else

                    t+=1

                    hst, lst, hjt, ljt = update_beliefs(propositor, hk_next, hm_next, lk_next, lm_next)
                end

            elseif(policy==2.0)

                t+=1

                hst, lst, hjt, ljt = update_beliefs(propositor, hk_next, hm_next, lk_next, lm_next)



            else      
                #(policy==1.0)

                observed_proposals += 1.0

                payoff_respondent, answer = answer_liq(m_L, t, hm_next, payoff_prop)

                if(answer==1.0)

                    result[prop_index] = payoff_prop
                    result[respondent_index] = payoff_respondent
                    result[3] = 1.0
                    result[4] = t
                    result[5] = observed_proposals

                else

                    result[prop_index] = payoff_prop
                    result[respondent_index] = payoff_respondent
                    result[3] = 3.0
                    result[4] = t
                    result[5] = observed_proposals


                end
            end


            if(t==T)

                result[1] = s_W[T, 1, 1, 1]#todos os valores finais de S são iguais, então acessarei o índice 1
                result[2] = j_W[T, 1, 1, 1]
                result[3] = 1.0
                result[4] = t
                result[5] = observed_proposals

            end





        end


        return result
    end
    
    function choose_parameters(propositor, hst, lst, hjt, ljt)
    
        if(propositor=="s")

            Pkt_array = Pst_array
            Cont_val = j_W
            prop_index = 1
            respondent_index = 2 #índice de j, para organizar o payoff
            m_L = j_L

            hkt = hst
            lkt = lst

            hmt = hjt
            lmt = ljt


            lk_next = hkt

            hk_next = draw_beta(hkt)
            hm_next = draw_beta(hmt)

        elseif(propositor=="j")
            Pkt_array = Pjt_array
            Cont_val = s_W
            prop_index = 2
            respondent_index = 1
            m_L = s_L

            hkt = hjt
            lkt = ljt

            hmt = hst
            lmt = lst


            lk_next = hkt


            hk_next = draw_beta(hkt)
            hm_next = draw_beta(hmt)

        else

            println("error: propositor not valid")

        end

        return Pkt_array, Cont_val, prop_index, respondent_index, m_L, hkt, lkt, hmt, lmt, lk_next, hk_next, hm_next
    end


    function draw_beta(hkt, β=game_parameters[2], grid=100)

        u = rand()

        if(hkt == 100)

            return 100

        else

            x = 1.0 - exp(1.0/β * (log(1.0 - u) + β * log(1.0 - hkt/grid)))
            x = round(x * 100, digits=0)
    #         return Int64(x * 100) #to convert in an integer
            return Int64(x)

        end
    end
    
    
    #cost function
    function Ct(t)
        #cost at period t=0(index1) is 0
        if(t <= 1)
            return 0
        else
            return c0 * D + c1 * (t-1) * D #test to make index==1 be t==0
        end
    end



    #liquidation payoffs
    function s_L(t)
        return min(L - Ct(t), Ds)
    end


    function j_L(t)
        return min(L - Ct(t) - s_L(t), Dj)
    end


    function proposal(Pkt_array, t, hkt, lmt)

        #pkt array será sempre do propositor, quem responder às propostas apenas olhará o seu valor de continuação

        return policy, payoff_prop, payment, lm_next = Pkt_array[t, hkt, lmt, [end,end-1, 1, 2]]

    end


    #maximum value of reorganization each period
    function Vt(Vmax, ρ, t)

        if(t <=1)
            return Vmax
        else
            #(t-2) instead of (t-1) because we shifted the indexes in the game so as to include t==0 at index==1
            return ρ^(t-2) * Vmax
        end


    end

    # answer_liq
    function answer_liq(m_L, t, hm_next, payoff_prop)

        liq = (m_L(t), Vt(Vmax, ρ, t) * hm_next/100 - payoff_prop)

        payoff_liq, answer = findmax(liq)

        return payoff_liq, answer
    end

    function answer_reorg(payment, Cont_val, t, hm_next, lm_next, lk_next)

        reorg_value = (payment, Cont_val[t+1, hm_next, lm_next, lk_next])

        payoff_reorg, answer = findmax(reorg_value)

        return payoff_reorg, answer
    end

    function update_beliefs(propositor, hk_next, hm_next, lk_next, lm_next)

        if(propositor=="s")
            hst = hk_next
            lst = lk_next

            hjt = hm_next
            ljt = lm_next

        else
            hst = hm_next
            lst = lm_next

            hjt = hk_next
            ljt = lk_next
        end


        return hst, lst, hjt, ljt
    end

    
    #simulation####
    simulated_moments = simulate_moments(S, N, θs0, θj0, μ, λj, s_W, j_W, Pst_array, Pjt_array)
    
    #inputed here manually
    data_moments = [0.85, 0.24, 2.78, 0.95, 1.00, 1.00, 0.75, 0.65]
    
    err = error_vector(data_moments, simulated_moments)
    
    #* in this case represents matrix multiplication
    criterion_val = transpose(err) * W_hat
    criterion_val = criterion_val * err
    
#     return (transpose(err) * W_hat) * err
    return criterion_val
    
end



    
    
    

criterion (generic function with 1 method)

Simulated Annealing

In [5]:
#function to generate random parameters
function mu_inv(y, mu)
    
    
    #entender porque tem 1 somando em mu
    #entender porque tem 1 subtraindo de abs(y)
    x = (((1 .+ mu) .^ abs.(y) .- 1)/mu) .* sign.(y)
    
    return x
    
end 

mu_inv (generic function with 1 method)

In [6]:
function compare_solutions(f0, g0, s0, fc, g, s, ftest, gtest, stest, Temp=Temp, TolFun=TolFun)
    
    
    #f0: function before the current loop
    #fc: function evaluated at the best current solution inside the loop
    #ftest: function evaluated at the test point inside the loop
    #gtest: game_parameters, test
    #stest: simulation_parameters, test
    
    #we evaluate the difference between the test point (ft) and the current point inside the loop(fc)
    #at the end, we compare the best between ftest and fc to the function before the loop (f0)
    
    df=ftest-fc

    #If the function variation,df, is <0 we take test point as current
    #point. And if df>0 we use Metropolis [5] condition to accept or
    #reject the test point as current point.
    #We use eps and TolFun to adjust temperature [4].   

    #retirei eps porque não estava definido antes. Cadê o cabo azul?
    if ((df < 0 || rand() < exp(-Temp*df/(abs(fc))/TolFun))==true)
        g = gtest
        s = stest
        fc = ftest 
    end

    #If the test point's solution (fx1) is better than current solution (fx), we take
    #current point as cuyrrent solution.       
    if ((fc < f0) ==true)
        g0 = gtest
        s0 = stest
        f0 = ftest
    end
    
    
    return g, s, fc, g0, s0, f0
    
end
    

compare_solutions (generic function with 3 methods)

In [16]:
function sim_anl(f, S, N, data, W_hat, g0, s0, lg, ug, ls, us, Mmax, ntests, TolFun)
    
    #algorithm adapted from 
    #https://www.mathworks.com/matlabcentral/fileexchange/33109-simulated-annealing-optimization
    
    #inputs####
    #f : function to evaluate
    #S: number of simulations per observation
    #N: number of observations in the data set
    #data = [Vh/D, L/D, Dj/D]
    #W_hat: weighting matrix for criterion function
    #g0: initial game parameters
    #s0: initial simulation parameters
    #lg and ug: lower and upper bounds for game parameters
    #ls and us: lower and upper bounds for simulation parameters
    #Mmax: maximum number of temperatures
    #ntest: number of test points simulated for each temperature
    #TolFun: parameter of tolerance to help choose the appropriate solutions
    
    
    #outputs####
    #the optimal values of the function and its parameters: f0, g0, s0
    #the best value of the function at each temperature and its parameters: fvals, gvals, svals
    
    
    
    
    if (size(g0,1)+size(s0,1))<6
        TolFun=1e-4;
        
    elseif (size(g0,1)+size(s0,1))<5
        Mmax=100;
    end

    
    
    
    #inicializando o algoritmo
    
    g = g0
    s = s0
    
    s_W, j_W, Pst_array, Pjt_array = solve_tree(data, g)
    
    fc = f(S, N, data, W_hat, g, s, s_W, j_W, Pst_array, Pjt_array)
    f0 = fc
    
    #para guardar os valores
    fvals = zeros(Mmax)
    gvals = zeros(Mmax, size(g, 1))
    svals = zeros(Mmax, size(s, 1))


    #Main loop simulates de annealing from a high temperature to zero in Mmax iterations.
    for m in 1:Mmax
        #We calculate T as the inverse of temperature.
        #Boltzman constant = 1
        Temp = m/Mmax
        mu=10^(Temp*100)


        #For each temperature we take 500 test points to simulate reach termal
        #equilibrium.
        for t1 in 1:ntests
            
            
            #we generate new gtest points using mu_inv function
            dg = mu_inv(2 .* rand(size(g,1)) .- 1, mu) .* (ug .- lg)
            
            dg = round.(dg, digits=3)
            
            gtest = g .+ dg
            
            #next step is to keep solution within bounds
            gtest = (gtest .< lg) .* lg .+ (lg .<= gtest) .* (gtest .<= ug) .* gtest .+ (ug .< gtest) .* ug
            
            #we evaluate the function and the change between the test point and the current point
            #for this we will have to solve the game, since we changed the game_parameters
            s_W, j_W, Pst_array, Pjt_array = solve_tree(data, gtest)
            
            #now we start another loop to make changes in s, the simulation parameters
            for t2 in 1:(ntests*10) #we will estimate 10 times more points here because the cost is lower
                
                
                #we generate new stest points using mu_inv function
                ds = mu_inv(2 .* rand(size(s,1)) .- 1, mu) .* (us .- ls)
                
                #we need to round because these are changes in θs0, θj0, which will be used as integers
                ds[1:2] .= round.(ds[1:2], digits=0)
                
                #ds[3] is μ, which can have 3 digits
                ds[3] = round(ds[3], digits=3)

                stest = s .+ ds

                #next step is to keep solution within bounds
                #this is just a multiplication using indicator functions
                stest = (stest .< ls) .* ls .+ (ls .<= stest) .* (stest .<= us) .* stest .+ (us .< stest) .* us
                

                
                #now we evaluate the criterion function directly
                #we don't need to solve the model again because we didn't change the game parameters
                
                #evaluation the criterion function
                #gtest = game_parameters in this test
                #stest = simulation parameters in this test

                ftest = f(S, N, data, W_hat, gtest, stest, s_W, j_W, Pst_array, Pjt_array)


                #comparing ftest with the current solution fc and the previous solution f0
                g, s, fc, g0, s0, f0 = compare_solutions(f0, g0, s0, fc, g, s, ftest, gtest, stest, Temp, TolFun)
                
            end
        end
        
        
        #record the value of the best solution of each cycle
        fvals[m] = f0
        gvals[m, :] .= g0
        svals[m, :] .= s0
            
        end
    
    return f0, g0, s0, fvals, gvals, svals
    
end





sim_anl (generic function with 2 methods)

In [7]:
#testing

f = criterion
S = 40
N = 75

#Vh/D, L/D, Dj/D
data = [1.0, 0.25, 0.68]; 

W_hat = Matrix{Int}(I, 8, 8);

#ρ, β, c0, λj
g0 = [0.884, 9.84, 0.044, 0.346];

#bounds for g
lg = [0.5, 1.0, 0.001, 0.1]
ug = [0.884, 12.0, 0.09, 0.9]


#θs0, θj0, μ
s0 = [28, 36, 4.566]

#bounds for s
#μ can be 0.5 month or 12 months
ls = [1, 1, 0.5]
us = [100, 100, 12]

Mmax = 2
ntests = 2
TolFun = 1e-4

# f0, g0, s0, fvals, gvals, svals = sim_anl(f, S, N, data, W_hat, g0, s0, lg, ug, ls, us, Mmax, ntests, TolFun)

0.0001

In [9]:
#testing sim_anl

if (size(g0,1)+size(s0,1))<6
    TolFun=1e-4;

elseif (size(g0,1)+size(s0,1))<5
    Mmax=100;
end

@show TolFun, Mmax

(TolFun, Mmax) = (0.0001, 2)

(0.0001, 2)

In [13]:
#initializing the algorithm

g = g0
s = s0

@show g, s;
@show data;

(g, s) = ([0.884, 9.84, 0.044, 0.346], [28.0, 36.0, 4.566])
data = [1.0, 0.25, 0.68]


In [14]:
s_W, j_W, Pst_array, Pjt_array = solve_tree(data, g)

 10.151322 seconds (36.02 M allocations: 4.572 GiB, 7.41% gc time)
 14.393886 seconds (37.51 M allocations: 7.974 GiB, 7.21% gc time)
 17.340070 seconds (41.72 M allocations: 10.762 GiB, 7.94% gc time)
 21.037476 seconds (44.98 M allocations: 13.114 GiB, 7.90% gc time)
 24.731415 seconds (47.56 M allocations: 15.033 GiB, 8.19% gc time)
 23.857199 seconds (49.48 M allocations: 16.526 GiB, 8.93% gc time)
 24.957897 seconds (51.28 M allocations: 17.957 GiB, 9.04% gc time)
 25.515958 seconds (52.51 M allocations: 18.965 GiB, 9.23% gc time)
 26.678222 seconds (53.44 M allocations: 19.739 GiB, 9.32% gc time)
 28.363365 seconds (54.32 M allocations: 20.479 GiB, 9.42% gc time)
 29.482496 seconds (54.96 M allocations: 21.026 GiB, 9.33% gc time)
 29.122500 seconds (55.60 M allocations: 21.562 GiB, 9.53% gc time)
 28.945907 seconds (56.52 M allocations: 22.420 GiB, 9.84% gc time)
304.787851 seconds (635.92 M allocations: 210.130 GiB, 8.90% gc time)


([0.22566048760788332 0.22888353905783249 … 0.8602210704138299 0.8762973987506573; 0.18267211317342694 0.18445554469396655 … 0.7920559676812302 0.8057438953015953; … ; 0.02599999998357369 0.025999999984129205 … 0.0308408652326135 0.0333973785521403; 0.01100000000000001 0.01100000000000001 … 0.01100000000000001 0.01100000000000001]

[0.0 0.22889429642163434 … 0.8602210704135698 0.8762973987509743; 0.0 0.1844596587513364 … 0.7920559676825571 0.8057438953031325; … ; 0.0 0.025999999999832758 … 0.0308408652535975 0.03339737857581836; 0.01100000000000001 0.01100000000000001 … 0.01100000000000001 0.01100000000000001]

[0.0 0.0 … 0.8602210704134008 0.8762973987509743; 0.0 0.0 … 0.7920559676825065 0.8057438953031325; … ; 0.0 0.0 … 0.0308408652535975 0.03339737857581836; 0.01100000000000001 0.01100000000000001 … 0.01100000000000001 0.01100000000000001]

...

[0.0 0.0 … 0.8602210630538759 0.8762973987509743; 0.0 0.0 … 0.7920559593215282 0.8057438953031325; … ; 0.0 0.0 … 0.0308408652535975 0.03339

In [43]:
#evaluating the criterion function
fc = f(S, N, data, W_hat, g, s, s_W, j_W, Pst_array, Pjt_array)
f0 = fc

11.871911575785582

In [44]:
#how much time to calculate the criterion function?
@btime f(S, N, data, W_hat, g, s, s_W, j_W, Pst_array, Pjt_array)

  19.442 ms (245183 allocations: 14.53 MiB)


11.876789396544668

In [45]:
#Can Julia infer correctly all data types? Not yet
@code_warntype f(S, N, data, W_hat, g, s, s_W, j_W, Pst_array, Pjt_array)

Variables
  #self#::Core.Compiler.Const(criterion, false)
  S::Int64
  N::Int64
  data::Array{Float64,1}
  W_hat::Array{Int64,2}
  game_parameters::Array{Float64,1}
  simulation_parameters::Array{Float64,1}
  s_W::Array{Float64,4}
  j_W::Array{Float64,4}
  Pst_array::Array{Float64,4}
  Pjt_array::Array{Float64,4}
  @_12::Int64
  θs0::Union{Float64, Int64}
  θj0::Union{Float64, Int64}
  μ::Float64
  ρ::Float64
  c0::Float64
  λj::Float64
  c1::Float64
  Vmax::Float64
  L::Float64
  Dj::Float64
  Ds::Float64
  D::Float64
  simulate_moments::var"#simulate_moments#35"{Array{Float64,4},Array{Float64,4},Array{Float64,4},Array{Float64,4}}
  loop_simulations@_26::Core.Box
  simulate_game@_27::Core.Box
  choose_parameters@_28::Core.Box
  draw_beta@_29::Core.Box
  Ct::var"#Ct#44"{Float64,Float64,Float64}
  s_L@_31::Core.Box
  j_L@_32::Core.Box
  proposal@_33::Core.Box
  Vt::var"#Vt#48"
  answer_liq@_35::Core.Box
  answer_reorg@_36::Core.Box
  update_beliefs@_37::Core.Box
  simulated_moments::Arr

│   %85  = %new(%82, s_W, j_W, Pst_array, Pjt_array, %83, %84, j_L@_32)::var"#choose_parameters#42"{Array{Float64,4},Array{Float64,4},Array{Float64,4},Array{Float64,4}}
│          Core.setfield!(choose_parameters@_28, :contents, %85)
│   %87  = Main.:(var"#draw_beta#43")::Core.Compiler.Const(var"#draw_beta#43", false)
│   %88  = Core.typeof(game_parameters)::Core.Compiler.Const(Array{Float64,1}, false)
│   %89  = Core.apply_type(%87, %88)::Core.Compiler.Const(var"#draw_beta#43"{Array{Float64,1}}, false)
│   %90  = %new(%89, game_parameters)::var"#draw_beta#43"{Array{Float64,1}}
│          Core.setfield!(draw_beta@_29, :contents, %90)
│   %92  = Main.:(var"#Ct#44")::Core.Compiler.Const(var"#Ct#44", false)
│   %93  = Core.typeof(c0)::Core.Compiler.Const(Float64, false)
│   %94  = Core.typeof(c1::Core.Compiler.Const(0.015, false))::Core.Compiler.Const(Float64, false)
│   %95  = Core.typeof(D)::Core.Compiler.Const(Float64, false)
│   %96  = Core.apply_type(%92, %93, %94, %95)::Core.Compile

In [49]:
#para guardar os valores
@show fvals = zeros(Mmax)
@show gvals = zeros(Mmax, size(g, 1))
@show svals = zeros(Mmax, size(s, 1));



fvals = zeros(Mmax) = [0.0, 0.0]
gvals = zeros(Mmax, size(g, 1)) = [0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0]
svals = zeros(Mmax, size(s, 1)) = [0.0 0.0 0.0; 0.0 0.0 0.0]


In [ ]:
#Main loop simulates de annealing from a high temperature to zero in Mmax iterations.
for m in 1:Mmax
    
    
    

In [554]:
Mmax = 100

100

In [555]:
#We calculate T as the inverse of temperature.
#Boltzman constant = 1

m = 1
Temp = m/Mmax

#since most of the data is limited in three decimal places, we want mu to have between 0 and 3 decimal places
mu=10^(Temp*100)

10.0

In [348]:
#For each temperature we take 500 test points to simulate reach termal
#equilibrium.
for t1 in 1:ntests
    
    
    

LoadError: syntax: incomplete: premature end of input

Studying mu_inv

In [139]:
function mu_inv(y,mu)

#     %This function is used to generate new point according to lower and upper
#     %and a random factor proportional to current point.
    x=(((1 .+ mu) .^ abs.(y) .- 1) ./ mu) .* sign.(y)
    
    
    return x

end

mu_inv (generic function with 1 method)

In [489]:
mu = 10^(1.0*10)

1.0e10

In [398]:
y = 2 * rand() - 1
mu = 10^(1/5)

x = ((1 + mu) ^ abs(y) - 1)/mu * sign(y) * (ug[2] - lg[2])

# sign(y)

5.767981401670105

In [581]:
#we generate new gtest points using mu_inv function
dg = mu_inv(2 .* rand(size(g,1)) .- 1, mu) .* (ug .- lg)
dg = round.(dg, digits=3)

4-element Array{Float64,1}:
  0.197
 -0.924
  0.005
 -0.602

In [333]:
gtest = g .+ dg

4-element Array{Float64,1}:
  0.7129909647634431
 10.391981217193687
  0.06439029566090873
  0.278266908080774

In [334]:
#next step is to keep solution within bounds
gtest = (gtest .< lg) .* lg .+ (lg .<= gtest) .* (gtest .<= ug) .* gtest .+ (ug .< gtest) .* ug

4-element Array{Float64,1}:
  0.7129909647634431
 10.391981217193687
  0.06439029566090873
  0.278266908080774

In [335]:
#we evaluate the function and the change between the test point and the current point
#for this we will have to solve the game, since we changed the game_parameters
s_W, j_W, Pst_array, Pjt_array = solve_tree(data, gtest)

 14.223692 seconds (42.07 M allocations: 10.940 GiB, 13.83% gc time)
 24.129928 seconds (48.85 M allocations: 16.062 GiB, 12.04% gc time)
 28.398924 seconds (53.06 M allocations: 19.089 GiB, 12.12% gc time)
 31.356015 seconds (55.27 M allocations: 20.774 GiB, 12.31% gc time)
 32.048500 seconds (56.79 M allocations: 21.962 GiB, 12.50% gc time)
 33.034318 seconds (57.46 M allocations: 22.919 GiB, 12.72% gc time)
163.215162 seconds (313.50 M allocations: 111.746 GiB, 12.49% gc time)


([0.22623763989613865 0.22836653725479605 … 0.9116764671020214 0.9214259136893708; 0.16424673466892342 0.16518597263225077 … 0.7891412743012818 0.7991766414826916; … ; 0.1106075528059098 0.11060808219611726 … 0.11561058642856177 0.11817399185688529; 0.09560970433909127 0.09560970433909127 … 0.09560970433909127 0.09560970433909127]

[0.0 0.2283885241496651 … 0.911676879942855 0.9214263774640035; 0.0 0.16518843033510636 … 0.7891428660641248 0.7991784297919041; … ; 0.0 0.1106097043390567 … 0.11561284051560491 0.1181765252051514; 0.09560970433909127 0.09560970433909127 … 0.09560970433909127 0.09560970433909127]

[0.0 0.0 … 0.911676879942855 0.9214263774640035; 0.0 0.0 … 0.7891428660641248 0.7991784297919041; … ; 0.0 0.0 … 0.11561284051560491 0.1181765252051514; 0.09560970433909127 0.09560970433909127 … 0.09560970433909127 0.09560970433909127]

...

[0.0 0.0 … 0.9116765514476293 0.9214263774640035; 0.0 0.0 … 0.7891426309802857 0.7991784297919041; … ; 0.0 0.0 … 0.11561284051560491 0.11817652

In [336]:
#checando quantos períodos esse jogo gerou
tcurioso = size(s_W,1)

7

In [ ]:
#now we start another loop to make changes in s, the simulation parameters
for t2 in 1:ntests

In [337]:
t2 = 1

1

In [338]:
#we generate new stest points using mu_inv function
ds = mu_inv(2 .* rand(size(s,1)) .- 1, mu) .* (us .- ls)

#we need to round because these are changes in θs0, θj0, which will be used as integers
ds[1:2] .= round.(ds[1:2], digits=0)

2-element view(::Array{Float64,1}, 1:2) with eltype Float64:
  13.0
 -31.0

In [339]:
stest = s .+ ds

#next step is to keep solution within bounds
#this is just a multiplication using indicator functions
stest = (stest .< ls) .* ls .+ (ls .<= stest) .* (stest .<= us) .* stest .+ (us .< stest) .* us

3-element Array{Float64,1}:
 41.0
  5.0
  0.5

In [340]:
#now we evaluate the criterion function directly
#we don't need to solve the model again because we didn't change the game parameters

#evaluation the criterion function
#gtest = game_parameters in this test
#stest = simulation parameters in this test

ftest = f(S, N, data, W_hat, gtest, stest, s_W, j_W, Pst_array, Pjt_array)

16.699585223524007

In [341]:
#comparing ftest with the current solution fc and the previous solution f0
g, s, fc, g0, s0, f0 = compare_solutions(f0, g0, s0, fc, g, s, ftest, gtest, stest, Temp, TolFun)

([0.884, 9.84, 0.044, 0.346], [28.0, 36.0, 4.566], 11.871911575785582, [0.884, 9.84, 0.044, 0.346], [28.0, 36.0, 4.566], 11.871911575785582)

In [ ]:
    end
end


#record the value of the best solution of each cycle
fvals[m] = f0
gvals[m, :] .= g0
svals[m, :] .= s0